# 데이터 읽기

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
dataset = pd.read_csv('Data.csv')
dataset.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [5]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

# 데이터 전처리

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [7]:
# 특성 정규화 작업
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
# SVM 구현
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

SVC(random_state=0)

# 모델 평가

### k-폴드 교차 검증

In [9]:
from sklearn.model_selection import cross_val_score
# 동시에 10개의 테스트 폴드에 배치한 모델들이 가져오는 10개의 정확도를 반환.
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
# 목록 안에 이는정화도 10개의 평균을 계산
print('Accuracy: {:.2f} %'.format(accuracies.mean()*100))
print('SD: {:.2f} %'.format(accuracies.std()*100))


Accuracy: 97.07 %
SD: 2.18 %


k-폴드 교차검증은 97.07로 전자가 더 좋아보일 수 있다.


그러나, 테스트를 한 번해서 나온 결과값이기 때문에 10개의 다른 테스트 세트인 후자가 더 신뢰도가 있을 수 있다.

### 그리드 검색

In [10]:
from sklearn.model_selection import GridSearchCV

# C파라미터: 정규화 파라미터로 과적합을 줄여서 훈련 성능을 향상시키도록 튜닝 c의 기본값은 1로 정규화와 반비례
# kernel: 커널 SVM모델 중 좋은 커널을 찾는 것 (ex: linear, rbf, sigmoid, poly)
# gamma: rbf, ploy, sigmoid를 사용할 때만 넣을 수 있는 파라미터

parameters = [{'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear']},
              {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)

# grid search는 훈련 세트에만 적용해야한다. 테스트 세트는 모델이나 튜닝된 모델을 테스트 할 때를 대비해 분리한 것이다. 

grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

Best Accuracy: 97.27 %
Best Parameters: {'C': 0.5, 'kernel': 'linear'}


그리드 검색에서 C가 0.5, 감마가 0.6, kernel이 rbf인 경우가 97.27%로 가장 높은 정확도가 나온것을 볼 수 있다.